In [4]:
#BILATERAL FILTERING

#Importing Libraries
import cv2
import numpy

#Defining Gaussian Filter
def gaussian(x,sigma):
    return (1.0/(2*numpy.pi*(sigma**2)))*numpy.exp(-(x**2)/(2*(sigma**2)))

#Defining Euclidean Distance between two points formula 
def distance(x1,y1,x2,y2):
    return numpy.sqrt(numpy.abs((x1-x2)**2+(y1-y2)**2))


def bilateral_filter(image, diameter, sigma_i, sigma_s):
    new_image = numpy.zeros(image.shape)#Defining New Image

    for row in range(len(image)):
        for col in range(len(image)):
            wp_total = 0
            filtered_image = numpy.zeros(1)
            for k in range(diameter):
                for l in range(diameter):
                    n_x =row - (diameter/2 - k)
                    n_y =col - (diameter/2 - l)
                    if n_x >= len(image):
                        n_x -= len(image)
                    if n_y >= len(image[0]):
                        n_y -= len(image[0])
                    gi = gaussian(abs(image[int(n_x)][int(n_y)] - image[row][col]), sigma_i)
                    gs = gaussian(distance(n_x, n_y, row, col), sigma_s)
# Calculating Weights Wp = q∈S Gσs(||p − q||) * Gσr(|Ip − Iq|)
                    wp = gi * gs
# Calculating q∈S Gσs(||p − q||) * Gσr(|Ip − Iq|)Iq
                    filtered_image = (filtered_image[0]) + (image[int(n_x)][int(n_y)] * wp)
                    wp_total = wp_total + wp
            filtered_image = filtered_image / wp_total
            new_image[row][col] = int(numpy.round(filtered_image[0]))
    return new_image

#Reading The Image
image = cv2.imread("in_img.jpg")

if image is not None:
    filtered_image_OpenCV = bilateral_filter(image, 3, 1000, 1000)
#Saving the Filtered Image
    cv2.imwrite("filtered_image.png", filtered_image_OpenCV)
else:
    print("Error: Unable to load the image.")

In [3]:
#Importing Libraries
import cv2
import numpy as np
from scipy.ndimage import convolve

#Taking Sample Intensities
intensities = [168,151,118]
intensities_1 = [[168],[151],[118]]

#image = np.array([[100,200],[300,400]])
                                 
input_image = cv2.imread("in_img.jpg")
image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

#Defining Euclidean Distance Formula between two points  
def distance(x1, y1, x2, y2):
    return ((x1 - x2)**2 + (y1 - y2)**2)

#Defining the spatial and range standard  Deviation Values
sigma_r = 10
sigma_s = 10

#Extracting the Layers
layers =[(1/((sigma_r**2)*2*np.pi)) * np.exp(((-0.5)*((image - intensity)**2))/(sigma_r**2)) * image for intensity in intensities]

#Defining range_kernel
range_kernel=[(1/((sigma_r**2)*2*np.pi)) * np.exp(((-0.5)*((image - intensity)**2))/(sigma_r**2)) for intensity in intensities_1]

d = []
for i in range(len(intensities_1)):
    for j in range(len(intensities_1[i])):
        spatial = []
        for row in range((image.shape[0])):
            for col in range((image.shape[1])):
                spatial.append(distance(i, j, row, col))
        d.append(spatial)

for i in range(len(d)):
    d[i] = sum(d[i])

#Defining spatial_kernel
spatial_kernel = [1/((sigma_s**2)*2*np.pi)*(np.exp(((-0.5)*(i**2))/(sigma_s**2))) for i in d]                                                 
                                                 
normalized_layers = []
i = 0
j = 0

#Convolution Operation
for layer in layers:
    convolved_layer = convolve(layer.astype(float),np.full((3,3),spatial_kernel[i]))
    normalized_layer = convolved_layer / convolve(range_kernel[i],np.full((2,2),spatial_kernel[i]))
    normalized_layers.append(normalized_layer)
    i+=1
    j+=1

                                                 
upsampled_layers = [np.repeat(np.repeat(layer, 2, axis=0), 2, axis=1) for layer in normalized_layers]

result = np.zeros_like(image)

for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        pixel_intensity = image[i, j]
        closest_values = sorted(intensities, key=lambda x: abs(x - pixel_intensity))[:2]
        k1, k2 = upsampled_layers[intensities.index(closest_values[0])], upsampled_layers[intensities.index(closest_values[1])]
        result[i, j] = ((((pixel_intensity - closest_values[0]) / (closest_values[1] - closest_values[0])) * k2[i, j]) 
                        + (((closest_values[1] - pixel_intensity) / (closest_values[1] - closest_values[0])) * k1[i, j]))

C:\Users\ACS\AppData\Local\Temp\ipykernel_1720\3236713544.py:51: RuntimeWarning: invalid value encountered in divide
  normalized_layer = convolved_layer / convolve(range_kernel[i],np.full((2,2),spatial_kernel[i]))
C:\Users\ACS\AppData\Local\Temp\ipykernel_1720\3236713544.py:66: RuntimeWarning: invalid value encountered in cast
  result[i, j] = ((((pixel_intensity - closest_values[0]) / (closest_values[1] - closest_values[0])) * k2[i, j])
